En este script obtenemos las respuestas de los clickbaits con contenido empleando el modelo basado en NER (usando Stanza). Buscamos la entidad esperada como respuesta en el titular y después extraemos la primera entidad del contenido que se corresponda con la esperada.

In [ ]:
# Instalar y cargar librerías necesarias
!pip install stanza pandas
import pandas as pd
import stanza

# Descargar modelo de español de Stanza
stanza.download('es')
nlp = stanza.Pipeline('es', processors='tokenize,ner')

# Cargar el CSV desde ruta local
file_path = 'dataset_clickbait_QA_limpio.csv'
df = pd.read_csv(file_path)

# Funciones auxiliares
import re

def infer_expected_entity(title):
    title = title.lower()

    # Patrones agrupados por tipo de entidad
    patterns = {
        "PER": [
            r"\bquién(es)?\b", r"\bfamos[oa]s?\b", r"\bperson(a|aje|alidad)(es)?\b",
            r"\bnombre(s)?\b", r"\bcelebridad(es)?\b", r"\bfigura(s)? pública(s)?\b"
        ],
        "LOC": [
            r"\bdónde\b", r"\blugar(es)?\b", r"\bciudad(es)?\b", r"\bpaís(es)?\b",
            r"\bubicación(es)?\b", r"\bregión(es)?\b", r"\bprovincia(s)?\b", r"\bsitio(s)?\b"
        ],
        "DATE": [
            r"\bcuándo\b", r"\bfecha(s)?\b", r"\bd[íi]a(s)?\b", r"\bañ[oa]s?\b",
            r"\bépoca(s)?\b", r"\btemporada(s)?\b", r"\bmomento(s)?\b", r"\bhora(s)?\b"
        ],
        "NUM": [
            r"\bcu[aá]nt[ao]s?\b", r"\bcantidad(es)?\b", r"\bn[uú]mer[oa]s?\b",
            r"\bcifra(s)?\b", r"\bporcentaje(s)?\b", r"\bdinero\b", r"\bvalor(es)?\b",
            r"\btotal(es)?\b", r"\bingreso(s)?\b", r"\bpes[o|os]\b", r"\beuro(s)?\b"
        ]
    }

    for entity, regex_list in patterns.items():
        for pattern in regex_list:
            if re.search(pattern, title):
                return entity

    return None


def extract_entity(text, expected_label):
    doc = nlp(text)
    for sent in doc.sentences:
        for ent in sent.ents:
            if ent.type == expected_label:
                return ent.text
    return None

# Generar la columna answer
answers = []
for i, row in df.iterrows():
    expected = infer_expected_entity(row['title'])
    if expected:
        answer = extract_entity(row['content'], expected)
    else:
        answer = None
    answers.append(answer)

df['answer'] = answers

# Filtrar solo las noticias con respuesta detectada
df = df[df['answer'].notnull()].reset_index(drop=True)
print(f"Número de filas tras filtrar respuestas detectadas: {len(df)}")

# Exportar como nuevo CSV
output_file = "qa_output_ner_stanza.csv"
df.to_csv(output_file, index=False)
print(f"Archivo guardado en: {output_file}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 47.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: es (Spanish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |
| ner       | conll02  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Número de filas tras filtrar respuestas detectadas: 29
Archivo guardado en: dataset_con_respuestas_stanza.csv
